In [ ]:
import numpy as np

import matplotlib.pyplot as pl
import matplotlib as mpl

pl.style.use('fivethirtyeight')


#mpl.rc('figure', figsize=(8,6))

#mpl.rc('figure', figsize=(14,12))
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'


font = {'size'   : 13}

mpl.rc('font', **font)

mpl.rcParams['lines.linewidth']=3
mpl.rcParams['axes.labelsize']=16


import os
import re

from scipy.io import loadmat
from scipy.ndimage import gaussian_filter1d
from scipy.signal import welch

import scipy.signal as sg

from functools import partial


In [ ]:
data_folder='../../../Data/AS-2022_02_22-ChinQ329_fmaskedCAP_normal'

listFiles = os.listdir(data_folder)

listFilesMat=[]

for filename in listFiles:
    if re.match('.*.mat', filename):
        listFilesMat.append(filename)
listFilesMat.sort()

### Functions

In [ ]:
#import inds
from Q329_inds import dic_inds, dic_ref_maskers, labels_gradual_amp

begin_ind, end_ind=dic_inds[40][0]  #take 1 tuple of begin_ind, end_ind to have list of maskers
pic_numbers_ignore=[]

In [ ]:
def split_name(filename):
    '''returns (picNumber, masker filename)'''
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', filename)
    assert m, f'{filename} did not match'
    p, name= m.groups()
    return int(p), name

map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', fln)
    if m:
        p, name = split_name(fln)

        if name not in map_table:
            map_table[name]=[p]
        else:
            li=map_table[name]
            li.append(p)
            map_table[name]=li
        
filtered_map_table={}  #map masker filename -> pic numbers
for fln in listFilesMat:
    
    
    m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', fln)
    if m:
        p, name = split_name(fln)
        if p>=begin_ind and p<=end_ind and p not in pic_numbers_ignore:

            if name not in filtered_map_table:
                filtered_map_table[name]=[p]
            else:
                li=filtered_map_table[name]
                li.append(p)
                filtered_map_table[name]=li

In [ ]:
sorted(list(filtered_map_table.keys()))

In [ ]:
def loadPic(n):
    found=False
    for filename in listFilesMat:
        if f'p{str(n).zfill(4)}' in filename:
            found=True
            break
    assert found, f'pic {n} not found.'
    arr= loadmat(f'{data_folder}/{filename}')
    return arr['data_struct']

def get_info_pic(pic):
    #avg
    arr=pic['AD_Data'][0][0]['AD_Avg_V'][0][0][0]
    
    #HACK only C or R
    '''
    all_data=pic['AD_Data'][0][0]['AD_All_V'][0][0]
    arr=np.zeros_like(all_data[0])
    for i in range(len(all_data)):
        if i%2==1:
            arr+=all_data[i]
    arr/=len(all_data)/2
    '''

    return {'arr':arr,
            'XstartPlot_ms':pic['Stimuli'][0][0]['CAP_intervals'][0][0]['XstartPlot_ms'][0][0][0],
            'XendPlot_ms': pic['Stimuli'][0][0]['CAP_intervals'][0][0]['XendPlot_ms'][0][0][0],
           'name':pic['Stimuli'][0][0]['masker'][0][0]['name'][0][0],
           'masker_atten':pic['Stimuli'][0][0]['masker_atten_dB'][0][0][0][0]}


    


check the consistency of attenuations

In [ ]:

def plotPic(picDic=None, n=None, label='', tmin=0, baselineSig=None):
    '''
    Plot pic given picture dictionary or pic number. 
    Args:
        baseline: Picture dictionary for baseline signal'''
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    pic_info=get_info_pic(picDic)
    if label=='':
        label=pic_info['name']
        
    
    val=pic_info['arr']
    
    t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))
    if baselineSig is not None:
        val=val-baselineSig
    #val=np.cumsum(val) #HACK!!
    pl.plot(t, val, label=label)
    pl.xlim([tmin,  pic_info['XendPlot_ms']])
    
def getMaskerSig(dic, name):
    
    picNums=dic[name]
    firstPic=True
    for picNum in picNums:
        picDic=loadPic(picNum)
        
    
        pic_info=get_info_pic(picDic)
    
        if firstPic:
            #val=np.sum(picDic['valAll'][1::2], axis=0)
            val=pic_info['arr']
            
            firstPic=False
        else:
            #val+=np.sum(picDic['valAll'][1::2], axis=0)
            val+=pic_info['arr']
    val/=len(picNums)
    return val
    
def plotMasker(dic, name, tmin=5, tmax=11, use_ref=False, baseline=None, corr_drift=False):
    '''
    Plot avg signal for masker given masker name and dictionary of picNums 
    Args:
        use_ref: use default ref masker
        baseline: signal for baseline signal (if use_ref False)
        corr_drift: not set for this expe'''
    #HACK correct labels for gradualamp hp6200
    if name in labels_gradual_amp:
        label=labels_gradual_amp[name]
    else:
        label=name
    val=getMaskerSig(dic, name)
    
    picNums=dic[name]
    for picNum in picNums:
        picDic=loadPic(picNum)
        pic_info=get_info_pic(picDic)
        
        break

    t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))
    
    if use_ref:
        ref_name=dic_ref_maskers[name]
        val_ref=getMaskerSig(dic, ref_name)
        val=val-val_ref
    
    elif baseline is not None:
        val=val-baseline
        
    #val=gaussian_filter1d(val, 0.02/(t[1]-t[0]))  #HACK filter
    #HACK
    #val=np.cumsum(val) #HACK!!
    #val-=val[300] #HACK!!
    pl.plot(t, val*1e3, label=label)
    if tmax==0:
        tmax=pic_info['XendPlot_ms']
    pl.xlim([tmin, tmax])
    pl.xlabel('t (ms)')
    if baseline is None:
        pl.ylabel('Amplitude (µV)')
    else:
        
        pl.ylabel('Amplitude difference (µV)')
    
'''
def getMaskerAtten(picDic=None, n=None):
    if not n is None:
        picDic=loadPic(n)
    assert not picDic is None
    ans=picDic['masker_atten_dB']
    return ans[0][0]
'''

\[new\]: find click artifacts 

In [ ]:

def findPicsArtifacts(begin_ind, end_ind, t_min=4e-3, t_max=5.2e-3, amp_min=80e-6):
    
    res=[]
    for n in range(begin_ind, end_ind+1):
        picDic=loadPic(n)
        pic_info=get_info_pic(picDic)

        val=pic_info['arr']
        t=np.linspace(pic_info['XstartPlot_ms'], pic_info['XendPlot_ms'], num=len(val))

        ind_t_min=np.sum(t<t_min*1e3)
        ind_t_max=np.sum(t<t_max*1e3)
        if np.any(np.abs(val[ind_t_min:ind_t_max])*1e-3>amp_min):
            res.append(n)

    return res

In [ ]:
#findPicsArtifacts(begin_ind, end_ind)  #t_min=4e-3, t_max=5.2e-3, amp_min=80e-6

#returns 71

In [ ]:
plotPic(n=71)

### Choice atten

In [ ]:
atten = 30

In [ ]:
def betweenInds(n, list_inds):
    '''
    Args:
        list_inds: list of tuples (begin_ind, end_ind)
    '''
    for begin_ind, end_ind in list_inds:
        if begin_ind<= n and n<=end_ind: 
            return True
    return False
        

def getFilteredMapTable(atten):
    filtered_map_table_atten={}
    for fln in listFilesMat:


        m=re.match('p([0-9]{4})_fmasked_CAP_(.*).mat', fln)
        if m:
            p, name = split_name(fln)
            if betweenInds(p, dic_inds[atten]) and p not in pic_numbers_ignore:

                if name not in filtered_map_table_atten:
                    filtered_map_table_atten[name]=[p]
                else:
                    li=filtered_map_table_atten[name]
                    li.append(p)
                    filtered_map_table_atten[name]=li
    return filtered_map_table_atten
    

filtered_map_table_atten=getFilteredMapTable(atten)  #map masker filename -> pic numbers


In [ ]:
dic=filtered_map_table_atten

### I/O func

In [ ]:
plotMasker(dic,'1_hp6000_narrowband5kHz_45dB')
plotMasker(dic, '2_hp6000_narrowband5kHz_40dB')
plotMasker(dic, '3_hp6000_narrowband5kHz_35dB')
plotMasker(dic, '4_hp6000_narrowband5kHz_32dB')
plotMasker(dic, '5_hp6000_narrowband5kHz_29dB')
plotMasker(dic, '6_hp6000_narrowband5kHz_26dB')
plotMasker(dic, '7_hp6000_narrowband5kHz_23dB')
plotMasker(dic, '8_hp6000_narrowband5kHz_20dB')
plotMasker(dic, '9_hp6000_narrowband5kHz_17dB')
plotMasker(dic,'10_hp6000_narrowband5kHz_14dB')

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
ref_masker_name=dic_ref_maskers['1_hp6000_narrowband5kHz_45dB']
print(f'ref masker: {ref_masker_name}')
plotMasker(dic,'1_hp6000_narrowband5kHz_45dB', use_ref=True)
plotMasker(dic, '2_hp6000_narrowband5kHz_40dB', use_ref=True)
plotMasker(dic, '3_hp6000_narrowband5kHz_35dB', use_ref=True)
plotMasker(dic, '4_hp6000_narrowband5kHz_32dB', use_ref=True)
plotMasker(dic, '5_hp6000_narrowband5kHz_29dB', use_ref=True)
plotMasker(dic, '6_hp6000_narrowband5kHz_26dB', use_ref=True)
plotMasker(dic, '7_hp6000_narrowband5kHz_23dB', use_ref=True)
plotMasker(dic, '8_hp6000_narrowband5kHz_20dB', use_ref=True)
plotMasker(dic, '9_hp6000_narrowband5kHz_17dB', use_ref=True)
plotMasker(dic,'10_hp6000_narrowband5kHz_14dB', use_ref=True)
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
pl.figure(figsize=(8, 6))
cap=[]
rms=[]
masker_list=[ '1_hp6000_narrowband5kHz_45dB',
 '2_hp6000_narrowband5kHz_40dB',
 '3_hp6000_narrowband5kHz_35dB',
 '4_hp6000_narrowband5kHz_32dB',
 '5_hp6000_narrowband5kHz_29dB',
 '6_hp6000_narrowband5kHz_26dB',
 '7_hp6000_narrowband5kHz_23dB',
 '8_hp6000_narrowband5kHz_20dB',
 '9_hp6000_narrowband5kHz_17dB',
  '10_hp6000_narrowband5kHz_14dB']
           
for masker in masker_list:
    sig=getMaskerSig(dic,masker)
    sig=sig[300:]
    ref_sig=getMaskerSig(dic, dic_ref_maskers[masker])
    ref_sig_trunc=ref_sig[300:]
    cap.append(np.max(sig-ref_sig_trunc)-np.min(sig-ref_sig_trunc))
    rms.append(np.std(sig-ref_sig_trunc))
    
    
#HACK

cap[-2]=-cap[-2]
cap[-1]=-cap[-1]

attens=-atten-np.array([45,40,35,32,29,26,23, 20, 17, 14])
pl.plot(attens, cap, label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('- Attenuation')
pl.ylabel('Amplitude')

### Hp 6000 gradual amp

In [ ]:
plotMasker(dic,'11_hp6200_gradualamp')
plotMasker(dic, '12_hp6200_gradualamp')
plotMasker(dic, '13_hp6200_gradualamp')
plotMasker(dic, '14_hp6200_gradualamp')
plotMasker(dic, '15_hp6200_gradualamp')
plotMasker(dic, '16_hp6200_gradualamp')


pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

**Note:** every band does not have the same amplitude so maskers are not always comparable only given notch width

In [ ]:
ref_masker_name=dic_ref_maskers['11_hp6200_gradualamp']
print(f'ref masker: {ref_masker_name}')

plotMasker(dic,'11_hp6200_gradualamp', use_ref=True)
plotMasker(dic, '12_hp6200_gradualamp', use_ref=True)
plotMasker(dic, '13_hp6200_gradualamp', use_ref=True)
plotMasker(dic, '14_hp6200_gradualamp', use_ref=True)
plotMasker(dic, '15_hp6200_gradualamp', use_ref=True)
plotMasker(dic, '16_hp6200_gradualamp', use_ref=True)


pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

### Notch 5.3 kHz gradual amp

In [ ]:
plotMasker(dic,'17_notch5300_bw1000')
plotMasker(dic, '18_notch5300_bw1200')
plotMasker(dic, '19_notch5300_bw1400')
plotMasker(dic, '20_notch5300_bw1800_attn24')
plotMasker(dic, '21_notch5300_bw1800_nonotch')
plotMasker(dic, '22_notch5k')
plotMasker(dic, '23_notch4800_bw900')
plotMasker(dic, '24_notch4800_bw800')


pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
ref_masker_name=dic_ref_maskers['17_notch5300_bw1000']
print(f'ref masker: {ref_masker_name}')

plotMasker(dic,'17_notch5300_bw1000', use_ref=True)
plotMasker(dic, '18_notch5300_bw1200', use_ref=True)
plotMasker(dic, '19_notch5300_bw1400', use_ref=True)
plotMasker(dic, '20_notch5300_bw1800_attn24', use_ref=True)
plotMasker(dic, '21_notch5300_bw1800_nonotch', use_ref=True)
plotMasker(dic, '22_notch5k', use_ref=True)
plotMasker(dic, '23_notch4800_bw900', use_ref=True)
plotMasker(dic, '24_notch4800_bw800', use_ref=True)


pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

### I/O func at all atten

In [ ]:
pl.figure(figsize=(8, 6))

masker_list=[ '1_hp6000_narrowband5kHz_45dB',
 '2_hp6000_narrowband5kHz_40dB',
 '3_hp6000_narrowband5kHz_35dB',
 '4_hp6000_narrowband5kHz_32dB',
 '5_hp6000_narrowband5kHz_29dB',
 '6_hp6000_narrowband5kHz_26dB',
 '7_hp6000_narrowband5kHz_23dB',
 '8_hp6000_narrowband5kHz_20dB',
 '9_hp6000_narrowband5kHz_17dB',
  '10_hp6000_narrowband5kHz_14dB']
    
for atten in [45, 40,35,30,25,20]:
    cap=[]
    rms=[]
    
    dic_atten=getFilteredMapTable(atten)
    for masker in masker_list:
        sig=getMaskerSig(dic_atten,masker)
        sig=sig[300:]
        ref_sig=getMaskerSig(dic_atten, dic_ref_maskers[masker])
        ref_sig_trunc=ref_sig[300:]
        cap.append(np.max(sig-ref_sig_trunc)-np.min(sig-ref_sig_trunc))
        rms.append(np.std(sig-ref_sig_trunc))
 

    #HACK

    cap[-2]=-cap[-2]
    cap[-1]=-cap[-1]

    cap=np.array(cap)
    
    attens=-atten-np.array([45,40,35,32,29,26,23, 20, 17, 14])
    pl.plot(attens, cap*1e3, label=f'atten ref: {atten} dB')
    
    pl.scatter((-atten - 20,), (0,), linewidth=3)
    

    #pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
    pl.legend()
    pl.xlabel('- Attenuation')
    pl.ylabel('Amplitude difference (µV)')

Same thing but aligning I/O curves (+atten 20dB of one curve matched on previous curve)  
Assuming 5dB diff (and last atten: 17dB/14dB)

In [ ]:
pl.figure(figsize=(8, 6))

masker_list=[ '1_hp6000_narrowband5kHz_45dB',
 '2_hp6000_narrowband5kHz_40dB',
 '3_hp6000_narrowband5kHz_35dB',
 '4_hp6000_narrowband5kHz_32dB',
 '5_hp6000_narrowband5kHz_29dB',
 '6_hp6000_narrowband5kHz_26dB',
 '7_hp6000_narrowband5kHz_23dB',
 '8_hp6000_narrowband5kHz_20dB',
 '9_hp6000_narrowband5kHz_17dB',
  '10_hp6000_narrowband5kHz_14dB']

next_point=0
    
for atten in [45, 40,35,30,25,20]:
    cap=[]
    rms=[]
    
    dic_atten=getFilteredMapTable(atten)
    for masker in masker_list:
        sig=getMaskerSig(dic_atten,masker)
        sig=sig[300:]
        ref_sig=getMaskerSig(dic_atten, dic_ref_maskers[masker])
        ref_sig_trunc=ref_sig[300:]
        cap.append(np.max(sig-ref_sig_trunc)-np.min(sig-ref_sig_trunc))
        rms.append(np.std(sig-ref_sig_trunc))
 

    #HACK

    cap[-2]=-cap[-2]
    cap[-1]=-cap[-1]

    cap=np.array(cap)
    
    attens=-atten-np.array([45,40,35,32,29,26,23, 20, 17, 14])
    pl.plot(attens, (next_point+cap)*1e3, label=f'atten ref: {atten} dB')

    if next_point!=0:
        pl.scatter((-atten - 20,), (next_point*1e3,), linewidth=3)
    
    #myCircle = pl.Circle((-atten - 20, next_point), 0.3, )
    #pl.gca().add_artist(myCircle)

    #linear interpolation
    next_point= next_point+1/3*cap[-2]+2/3*cap[-1]
    
    #pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
    pl.legend()
    pl.xlabel('- Attenuation')
    pl.ylabel('Amplitude difference (µV)')

### notch 5.3 kHz all atten

In [ ]:
pl.figure(figsize=(8, 6))

masker_list=[
 '17_notch5300_bw1000',
 '18_notch5300_bw1200',
 '19_notch5300_bw1400']
    
for atten in [45, 40,35,30,25,20]:
    cap=[]
    rms=[]
    
    dic_atten=getFilteredMapTable(atten)
    for masker in masker_list:
        sig=getMaskerSig(dic_atten,masker)
        sig=sig[300:]
        ref_sig=getMaskerSig(dic_atten, dic_ref_maskers[masker])
        ref_sig_trunc=ref_sig[300:]
        cap.append(np.max(sig-ref_sig_trunc)-np.min(sig-ref_sig_trunc))
        rms.append(np.std(sig-ref_sig_trunc))
 


    bws=np.array([1000, 1200, 1400])
    cap=np.array(cap)
    pl.scatter(bws, cap*1e3, label=f'atten ref: {atten} dB')
    

    #pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
    pl.legend()
    pl.xlabel('Notch width (Hz)')
    pl.ylabel('Amplitude diff (µV, ref: no notch)')
    
    pl.ylim([0,2.5])